# 特徴量エンジニアリング（カテゴリ特徴量）

In [ ]:
!pip install -U pip -q
!pip install scikit-learn==0.23.1 -q
!pip install category_encoders -q
!git clone https://github.com/ironerumi/fe_workshop.git -q -b work

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from pandas import DataFrame, Series
pd.set_option('display.max_columns', 100)

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import category_encoders as ce

from google.colab import files

In [ ]:
# collaboratoryにLendingClub50000.csvをアップロードする。
#uploaded = files.upload() # ローカルからアップロードできるが今回は使用しない。

In [ ]:
# ファイルを読み込む
df = pd.read_csv('fe_workshop/dataset/LendingClub50000.csv')

In [5]:
df.head().T

,0,1,2,3,4
申込ID,1529851,1824764,403548,646411,552526
メンバーID,1793711,2126933,442721,799671,711946
年,2012,2012,2009,2011,2010
月,9,11,5,1,7
ローン申請額,16000,3600,19200,21000,4000
借り入れ目的（大分類）,debt_consolidation,debt_consolidation,wedding,credit_card,other
借り入れ目的（小分類）,Debt Consolidation / Final Wedding Exp,Crush Credit Cards,Consolidate debt and pay for wedding,Wells BofA Credit Card Refi,Debt Consolidation
勤務先,PwC,Morgan Stanley Smith Barney,Aggregate Knowledge,Emerson Process Management,Home Depot
勤続年数,3,2,1,1,1
居住形態,RENT,RENT,RENT,RENT,RENT


## One-hoeエンコーディング

In [9]:
# pandasのget_dummiesでOne-hot Encodingする
df[['グレード']].head().merge(pd.get_dummies(df['グレード']).head(), left_index=True, right_index=True)

,グレード,A,B,C,D,E,F,G
0,A,1,0,0,0,0,0,0
1,B,0,1,0,0,0,0,0
2,B,0,1,0,0,0,0,0
3,B,0,1,0,0,0,0,0
4,C,0,0,1,0,0,0,0


In [7]:
# sklearnのOneHotEncoderでOne-hot Encodingする
ohe = OneHotEncoder()
ohe.fit_transform(df[['グレード']]).A[:5, :] # np.arrayで返ってくる

array([[1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.]])

In [10]:
# category_encodersのOneHotEncoderでOne-hot Encodingする
ohe = ce.OneHotEncoder(handle_unknown='indicator', handle_missing=True, use_cat_names=True)
# 空→nan、未知の値→-1として対応
df[['グレード']].head().merge(ohe.fit_transform(df[['グレード']]).head(), left_index=True, right_index=True)

,グレード,グレード_A,グレード_B,グレード_C,グレード_E,グレード_F,グレード_D,グレード_G,グレード_nan,グレード_-1
0,A,1,0,0,0,0,0,0,0,0
1,B,0,1,0,0,0,0,0,0,0
2,B,0,1,0,0,0,0,0,0,0
3,B,0,1,0,0,0,0,0,0,0
4,C,0,0,1,0,0,0,0,0,0


In [12]:
# 仮に1行目の'grade'を未知のデータに置き換えると
df.loc[0,'グレード']='Z'
df[['グレード']].head().merge(ohe.transform(df[['グレード']]).head(), left_index=True, right_index=True) # -1に割り当てられる

,グレード,グレード_A,グレード_B,グレード_C,グレード_E,グレード_F,グレード_D,グレード_G,グレード_nan,グレード_-1
0,Z,0,0,0,0,0,0,0,0,1
1,B,0,1,0,0,0,0,0,0,0
2,B,0,1,0,0,0,0,0,0,0
3,B,0,1,0,0,0,0,0,0,0
4,C,0,0,1,0,0,0,0,0,0


In [13]:
# 1行目'grade'の値を戻します
df.loc[0,'グレード']='A'

In [14]:
# では、'addr_state'も同様に処理できるだろうか？
df[['住所（州）']].head()

,住所（州）
0,NJ
1,MD
2,CA
3,CA
4,GA


In [15]:
# ユニーク数の多い'addr_state'に対してOne-hotを行うとカラム数が膨大になってしまう。
ohe = ce.OneHotEncoder(use_cat_names=True)
df[['住所（州）']].head().merge(ohe.fit_transform(df[['住所（州）']]).head(), left_index=True, right_index=True)

,住所（州）,住所（州）_NJ,住所（州）_MD,住所（州）_CA,住所（州）_GA,住所（州）_NY,住所（州）_OH,住所（州）_IN,住所（州）_TX,住所（州）_FL,住所（州）_MA,住所（州）_AL,住所（州）_MN,住所（州）_NV,住所（州）_MO,住所（州）_IL,住所（州）_AR,住所（州）_SC,住所（州）_NC,住所（州）_PA,住所（州）_CO,住所（州）_DC,住所（州）_LA,住所（州）_SD,住所（州）_RI,住所（州）_AZ,住所（州）_MI,住所（州）_CT,住所（州）_WI,住所（州）_WA,住所（州）_KS,住所（州）_WV,住所（州）_OR,住所（州）_HI,住所（州）_VA,住所（州）_KY,住所（州）_OK,住所（州）_NH,住所（州）_MT,住所（州）_DE,住所（州）_NM,住所（州）_UT,住所（州）_VT,住所（州）_WY,住所（州）_ID,住所（州）_AK,住所（州）_MS,住所（州）_TN,住所（州）_IA,住所（州）_NE,住所（州）_ME
0,NJ,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,MD,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,CA,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,CA,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,GA,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Labelエンコーディング

In [16]:
# sklearnのLabelEncoderを用いる
le = LabelEncoder()
le.fit_transform(df[['住所（州）']].values.ravel())[:5]

array([30, 20,  4,  4, 10])

In [17]:
# category_encodersのOrdinalEncoderを用いる。
# 割り振られるラベルは異なるが基本的には同じ。
# ただし、DataRobotのOrdinalEncodingとは大きく異なる点に注意。
oe = ce.OrdinalEncoder()
df[['住所（州）']].head().merge(oe.fit_transform(df[['住所（州）']]).head(), left_index=True, right_index=True, suffixes=('','_変換後'))

,住所（州）,住所（州）_変換後
0,NJ,1
1,MD,2
2,CA,3
3,CA,3
4,GA,4


## Frequency/Countエンコーディング

In [18]:
# 州ごとの観測数をカウントする。
summary = df.groupby(['住所（州）'])[['住所（州）']].count()
summary.head()

,住所（州）
住所（州）,
AK,152
AL,617
AR,346
AZ,1097
CA,8707


In [19]:
# 集計結果をapplyする。
df[['住所（州）']].head().merge(df['住所（州）'].apply(lambda x: summary.loc[x]).head(), left_index=True, right_index=True, suffixes=('','_カウント'))

,住所（州）,住所（州）_カウント
0,NJ,2182
1,MD,1196
2,CA,8707
3,CA,8707
4,GA,1620


## Targetエンコーディング

In [20]:
# 'addr_state'について観測数の少ないカテゴリがどの程度か確認する。
df['住所（州）'].value_counts()

CA    8707
NY    4722
TX    3773
FL    3701
NJ    2182
IL    1933
PA    1796
VA    1630
GA    1620
OH    1523
MA    1424
NC    1273
MD    1196
AZ    1097
WA    1093
MI    1070
CO     986
CT     888
MO     834
MN     793
NV     685
AL     617
WI     597
LA     577
OR     575
SC     566
KS     415
OK     411
KY     411
AR     346
UT     338
NM     256
HI     255
RI     250
NH     234
DC     213
WV     208
AK     152
MT     139
DE     135
WY     114
VT      98
SD      92
IN      28
MS      15
TN      11
IA       9
ID       7
NE       3
ME       2
Name: 住所（州）, dtype: int64

In [21]:
# category_encodersのTargetEncoderを用いる。
# min_samples_leafはvalue_countsを参考に100にしてみた。
# 州記号を州ごとの平均貸し倒れ率に置換したことになる。
te = ce.TargetEncoder(min_samples_leaf=100)
df[['住所（州）']].head().merge(te.fit_transform(df[['住所（州）']], df[['貸し倒れ']]).head(), left_index=True, right_index=True, suffixes=('','_カウント'))

,住所（州）,住所（州）_カウント
0,NJ,0.156737
1,MD,0.155518
2,CA,0.155737
3,CA,0.155737
4,GA,0.166049


In [22]:
# TargetEncoderの対象は予測ターゲットに限定されない。
# ここでは'annual_inc'を選択したため、州記号を州ごとの平均年収に置換したことになる。

# 特にインパクトの大きい特徴量がある場合に有効で、予測ターゲットを用いておらず、ターゲットリーケージの懸念がほぼない点で有用。
te = ce.TargetEncoder(min_samples_leaf=50)
result = te.fit_transform(df[['住所（州）']], df[['年収']])
df[['住所（州）']].head().merge(result.head(), left_index=True, right_index=True, suffixes=('','_カウント'))

,住所（州）,住所（州）_カウント
0,NJ,75177.647562
1,MD,79444.082366
2,CA,72415.453038
3,CA,72415.453038
4,GA,69727.534377


In [23]:
# 州記号を州ごとの平均年収を採用してみる。
df['住所（州）'] = result['住所（州）']

In [ ]:
# 結果を保存する。
df.to_csv('LendingClub50000_ave_income_state.csv', index=False)

In [ ]:
# ダウンロードしたら、DataRobotでもう一度予測してみよう
files.download('LendingClub50000_ave_income_state.csv')